# リンク予測( Indication)
## ECFP -> ニューラルネットワーク

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

In [2]:
import torch
import torch.utils.data
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # 最適化アルゴリズム実装のためのライブラリ

In [3]:
df = pd.read_pickle("data_indication.pkl")

In [4]:
df.head()

,smiles1,smiles2,indication
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.000000
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.042896
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.000000


In [5]:
df.median()

indication    0.0
dtype: float64

In [6]:
def f(x):
    ans = 0
    if x > 0 and x <= 1:
        ans = 1
    else:
        ans = 0
    return ans 

In [7]:
df["indication"] = df["indication"].map(f)

In [12]:
class Net(nn.Module):  # 多層ニューラルネットワークの構築
    def __init__(self):
        super(Net, self).__init__()
        self.share1 = nn.Linear(512,1024)
        self.share2 = nn.Linear(1024,512)
        self.fc1 = nn.Linear(1024, 128)  # 一つ目の隠れ層のユニット数は512
        self.fc2 = nn.Linear(128, 128)  # 二つ目の隠れ層のユニット数は128
        self.fc3 = nn.Linear(128, 1)  # 出力層のユニット数は1


    def forward(self, x, y):
        a = F.relu(self.share1(x))
        a = self.share2(a)
        b = F.relu(self.share1(y))
        b = self.share2(b)
        x = torch.cat([a,b], dim=1)
        x = F.relu(self.fc1(x))  # 活性化関数にはReLUを使用
        x = F.relu(self.fc2(x))  # 活性化関数にはReLUを使用
        x = F.sigmoid(self.fc3(x))
        return x

net = Net()

In [18]:
df_sample = df.sample(n=10000)
df_sample = df_sample.reset_index(drop=True)

In [19]:
df_sample.head()

,smiles1,smiles2,indication
0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...",0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0


In [20]:
A = df_sample["smiles1"]
B = df_sample["smiles2"]
target = df_sample["indication"]

In [22]:
A = torch.Tensor(A)
B = torch.Tensor(B)
target = torch.Tensor(target)

In [23]:
train_tensor = torch.utils.data.TensorDataset(A,B,target)
train_dataset, test_dataset = torch.utils.data.random_split(train_tensor, [9000, 1000])

In [24]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [25]:
criterion = nn.BCELoss() 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)

In [26]:
for epoch in range(100):
    total_loss = 0.0
    for i,data in enumerate(train_loader):
        inputA,inputB, labels = data
        optimizer.zero_grad()  
        outputs = net(inputA,inputB)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(total_loss)
print('\n学習が終了しました。')

C:\Users\Takumi Inui\Anaconda\lib\site-packages\torch\nn\functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\Takumi Inui\Anaconda\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
C:\Users\Takumi Inui\Anaconda\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


144.33369435369968
101.74050308763981
99.94458747655153
99.75395145267248
99.77213903516531
99.41112932562828
99.4223319068551
99.26786535233259
98.70694889128208
98.50787936151028
98.67235188931227
98.02743844687939
97.74888534098864
97.63083467632532
96.99392611533403
96.47713728249073
96.03982254117727
95.34204337745905
94.690533824265
93.50935207307339
92.56752161681652
91.8165828064084
90.79663008451462
89.95665312558413
88.34247997403145
86.99688713997602
85.59344077855349
84.54317712038755
83.00112175941467
81.68432714045048
80.2738553211093
79.14953967183828
77.55126954615116
76.08289863169193
74.66173551604152
73.36139987036586
72.03173745423555
71.06669485569
69.82758870348334
68.74257405847311
67.47080155462027
66.40212348476052
65.77595895342529
64.95188376307487
63.591886296868324
63.33169973641634
63.061614360660315
61.86258800700307
61.77470628172159
60.43634118139744
59.84214677661657
59.41288078110665
58.3816523514688
58.17625864967704
57.603965155780315
57.26683987677

In [27]:
correct = 0.0  # 正答数を表す
total = 0.0  # テストデータの総数を表す
link = 0.0

In [28]:
y_label = np.empty(0)
y_predict = np.empty(0)

In [29]:
for data in test_loader:
    inputA, inputB, labels = data
    outputs = net(inputA,inputB)
    _, predicted = torch.max(outputs.data, 0)
    total += labels.size(0)  # テストデータの総数を計算
    correct += (predicted == labels).sum().item()  # 正答数を計算
    link += sum(labels.numpy())
    y_label = np.concatenate([y_label, np.array(labels.numpy())])
    y_predict = np.concatenate([y_predict, np.array(net(inputA,inputB).detach().numpy()).ravel()])

In [30]:
pred = []
for i in y_predict:
    if i >= 0.5:
        pred.append(1)
    else:
        pred.append(0)

In [31]:
count = 0
for i in range(1000):
    if y_label[i] == pred[i]:
        count += 1
count

826

In [32]:
print('テストデータに対する正答率： %d / %d = %f' % (count, total, count / total) + '\n')
print('テストデータに含まれるリンクありの割合： %d / %d = %f' % (link, total, link / total) + '\n')
print('テストデータに含まれるリンクなしの割合： %d / %d = %f' % (total - link, total, (total- link) / total) + '\n')

for i in range(10):  # テストデータの一部を10行に分けて可視化
    print("ラベル：" + "".join('%d ' % y_label[i*8 + j] for j in range(8)))  # ラベルの値を表示
    print("　予測：" + "".join('%d ' % pred[i*8 + j] for j in range(8)) + "\n")  # 予測結果を表示

テストデータに対する正答率： 826 / 1000 = 0.826000

テストデータに含まれるリンクありの割合： 111 / 1000 = 0.111000

テストデータに含まれるリンクなしの割合： 889 / 1000 = 0.889000

ラベル：0 0 0 0 0 0 0 0 
　予測：0 0 0 0 0 1 0 0 

ラベル：0 0 0 0 0 0 0 0 
　予測：0 0 0 0 0 0 0 0 

ラベル：0 0 0 0 0 0 0 0 
　予測：0 0 0 0 1 0 0 0 

ラベル：0 0 0 0 0 0 1 0 
　予測：0 0 1 0 0 0 0 1 

ラベル：0 0 0 0 0 0 0 1 
　予測：0 0 0 0 0 0 0 0 

ラベル：0 0 0 0 0 0 0 0 
　予測：0 0 0 0 0 0 0 0 

ラベル：0 0 0 0 0 0 0 0 
　予測：1 0 0 0 0 1 0 0 

ラベル：0 0 0 0 0 0 0 1 
　予測：0 0 0 0 0 0 0 1 

ラベル：0 0 0 0 0 1 0 0 
　予測：0 0 0 0 0 1 0 0 

ラベル：0 1 0 0 0 0 0 0 
　予測：0 1 0 0 0 0 0 0 



In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
roc_auc_score(y_label, y_predict)

0.6781990089076704